### Loading Data

In [ ]:
# load some packages
import matplotlib.pyplot as plt
import healpy as hp
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score

from MMD_functions import *

# load the catalogue
catalogue1000 = np.load('catalogue_1000sqd.npy')

In [ ]:
nside = 64 # HEALPix nside parameter

cl_kappa_225 = np.loadtxt('cl_kappa_mean_225.txt')[:,1]
cl_kappa_225 = np.concatenate((np.zeros(2), cl_kappa_225)) # add zeros for monopole and dipole
kappamap_225 = hp.synfast(cl_kappa_225, nside)
print("Cl_kappa225 shape:", cl_kappa_225.shape, "   kappamap225 shape:", kappamap_225.shape )
pixscale = 0.263
sizes_in_arcsec1000 = catalogue1000['r50'] * pixscale   #arcsec
print( "\nrange of DEC of catalogue1000:")
print(f"[{min(catalogue1000['dec'])},{max(catalogue1000['dec'])}]" )
print( "\nrange of RA of catalogue1000:")
print(f"[{min(catalogue1000['ra'])},{max(catalogue1000['ra'])}]" )


In [ ]:
# Convert galaxy coordinates to HEALPix pixel indices
galaxy_pix1000= hp.ang2pix(nside, catalogue1000['ra'], catalogue1000['dec'], lonlat=True)
galaxy_pix1000_unique, galaxy_pix1000_counts = np.unique(galaxy_pix1000, return_counts=True)
n_pixels = hp.nside2npix(nside)

print("Galaxy pixels:",galaxy_pix1000, "   Length of Galaxy pixels(should match nr. of galaxies):", len(galaxy_pix1000))
print("Total number of galaxies:", len(catalogue1000))
print("Total number of pixels:", n_pixels)
print("Number of unique pixels with galaxies:", len(galaxy_pix1000_unique))
print("Max number of galaxies in a pixel:", np.max(galaxy_pix1000_counts))
print("Min number of galaxies in a pixel:", np.min(galaxy_pix1000_counts))
print("Mean number of galaxies in a pixel:", np.mean(galaxy_pix1000_counts))
print("Number of pixels with > 20'000 galaxies:", np.sum(galaxy_pix1000_counts >= 20000))
print('-'*30)

In [ ]:
intrinsic_size1000 = sizes_in_arcsec1000
observed_size1000 = sizes_in_arcsec1000 * (1.0 + kappamap_225[galaxy_pix1000])

size_mask = (intrinsic_size1000 < 5.0) #arcsec


### Computing and Plotting the mean of each pixel
This is the most basic statistical measure, so we want to see, if we get a clear signal here.

Thresholds/Constraints from Noah's report:
- 20'000 galaxies per pixel for nside=64
- nside=64: In this case, the avg number of galaxies is above the needed threshold
- Outliers/Size threshold for too large galaxies: 5arcsec

In [ ]:
# Apply intrinsic size mask 
size_mask = (intrinsic_size1000 < 5.0) #arcsec

galaxy_pix1000_masked = galaxy_pix1000[size_mask]
intrinsic_size1000_masked = intrinsic_size1000[size_mask]
observed_size1000_masked = intrinsic_size1000_masked* (1.0 + kappamap_225[galaxy_pix1000_masked])
galaxy_pix1000_unique_masked, galaxy_pix1000_counts_masked = np.unique(galaxy_pix1000_masked, return_counts=True)

print("Summary after applying size cut of 5 arcsec:")
print("Galaxy pixels:",galaxy_pix1000_masked, "   Length of Galaxy pixels(should match nr. of galaxies):", len(galaxy_pix1000_masked))
print("Total number of galaxies:", len(intrinsic_size1000_masked))
print("Total number of pixels:", n_pixels)
print("Number of unique pixels with galaxies:", len(galaxy_pix1000_unique_masked))
print("Max number of galaxies in a pixel:", np.max(galaxy_pix1000_counts_masked))
print("Min number of galaxies in a pixel:", np.min(galaxy_pix1000_counts_masked))
print("Mean number of galaxies in a pixel:", np.mean(galaxy_pix1000_counts_masked))
print("Number of pixels with > 20'000 galaxies:", np.sum(galaxy_pix1000_counts_masked >= 20000))
print('-'*30)

In [ ]:
means_unlensed = []
means_lensed = []
pixels = []


for p in galaxy_pix1000_unique_masked:
    mask = (galaxy_pix1000_masked == p)
    if sum(mask) > 20000:   # Only consider pixels with more than 20'000 galaxies
        means_unlensed.append(np.mean(intrinsic_size1000_masked[mask]))
        means_lensed.append(np.mean(observed_size1000_masked[mask]))
        pixels.append(p)

means_unlensed = np.array(means_unlensed)
means_lensed = np.array(means_lensed)
# np.save("means_unlensed_1000sqd.npy", means_unlensed)
# np.save("means_lensed_1000sqd.npy", means_lensed)
# np.save("pixels_1000sqd.npy", np.array(pixels))

# means_lensed = np.load("means_lensed_1000sqd.npy")
# means_unlensed = np.load("means_unlensed_1000sqd.npy")
# pixels = np.load("pixels_1000sqd.npy")

In [ ]:
mean_theory = np.mean(intrinsic_size1000)*(1.0 + kappamap_225[pixels]) 
m, b = np.polyfit(kappamap_225[pixels], means_lensed,1)
print("Fit parameters (m,b):", m, b)
print("Mean intrinsic size:", np.mean(intrinsic_size1000))
# MMD^2 = (mean_pixel - mean_global)^2
mmd2_lensed_linear_exakt = (means_lensed - np.mean(observed_size1000_masked))**2


In [ ]:
plt.figure()
plt.plot(kappamap_225[pixels], means_unlensed, '.', label='Unlensed')
plt.plot(kappamap_225[pixels], means_lensed, '.', label='Lensed')
plt.plot(kappamap_225[pixels], mean_theory, '--', color='g', label=r'theory $[\theta_{obs}=\theta_{intr}(1+\kappa)]$')
plt.plot(kappamap_225[pixels], m*kappamap_225[pixels]+b, '--', color='black', label='fit')
plt.plot([np.min(kappamap_225[pixels]),np.max(kappamap_225[pixels])],[np.mean(intrinsic_size1000), np.mean(intrinsic_size1000)], '--', color='grey', label='mean intrinsic size')
plt.legend()
plt.xlabel(r'$\kappa$')
plt.ylabel('Mean size [arcsec]')
plt.title('Mean sizes in pixels with > 20\'000 galaxies')
plt.show()

In [ ]:
# Welche Pixel müssten noch ein Minuszeichen beim MMD erhalten?

plus_min_mask = (means_lensed - np.mean(observed_size1000_masked)) < 0  # 0 = positive, 1 = negative
plus_min_mask = (-2) * plus_min_mask.astype(int) + 1    # 1 = positive, -1 = negative
plus_min_mask

### Now use the MMD (linear kernel)

In [ ]:
# SIMPLE IMPLEMENTATION OF MMD FOR DIFFERENT KERNELS ---------------------------------
def compute_mmd(X, Y, kernel):
    """
    Compute Maximum Mean Discrepancy (MMD) between samples X and Y using a provided kernel.
    
    Parameters:
        X: array-like, shape (n_samples_X, n_features)
        Y: array-like, shape (n_samples_Y, n_features)
        kernel: callable, must support signature kernel(X, Y), returns kernel matrix
        
    Returns:
        mmd2: float, MMD^2 value
    """

    X = np.asarray(X)
    Y = np.asarray(Y)
    m = X.shape[0]
    n = Y.shape[0]
    
    K_XX = kernel(X, X)
    K_YY = kernel(Y, Y)
    K_XY = kernel(X, Y)
    
    # Remove diagonal for unbiased estimator
    # np.fill_diagonal(K_XX, 0)
    # np.fill_diagonal(K_YY, 0)
    
    mmd2 = (K_XX.sum() / (m * m)) \
        + (K_YY.sum() / (n * n)) \
        - (2 * K_XY.sum() / (m * n))
    
    return mmd2

def compute_mmd_subsample(X, Y, kernel, size_X=1000, size_Y=1000, n_iter=10, random_state=None):
    """
    Compute MMD between large X and Y by random subsampling.
    Parameters:
        X: array-like (N_X, features), large dataset
        Y: array-like (N_Y, features), large dataset
        kernel: callable kernel (scikit-learn compatible)
        size_X: int, subsample size from X
        size_Y: int, subsample size from Y
        n_iter: int, number of repetitions
        random_state: int or None, reproducibility
    Returns:
        avg_mmd: float, average MMD over n_iter subsamples
        mmd_values: list of individual MMD values
    """
    rng = np.random.default_rng(random_state)
    mmd_values = []
    for i in range(n_iter):
        Xs = rng.choice(X, size_X, replace=False)
        Ys = rng.choice(Y, size_Y, replace=False)
        mmd = compute_mmd(Xs, Ys, kernel)
        mmd_values.append(mmd)
    return np.mean(mmd_values)


Wir lassen mal für alle pixels mit > 20'000 Galaxien die MMD berechnen mit Linear Kernel. Es sollte ein paar Stunden dauern aber wir erwarten die gleichen Ergebnisse wie beim Mean.

In [ ]:
mmd2_lensed_linear_kernel = []
Y_lensed = observed_size1000_masked.reshape(-1,1)
for p in pixels:
    mask = (galaxy_pix1000_masked==p)
    if mask.sum()>20000:
        X = observed_size1000_masked[mask].reshape(-1,1)
        mmd2_lensed_linear_kernel.append(compute_mmd_subsample(X, Y_lensed, linear_kernel, 20000,20000,5,42))

mmd2_lensed_linear_kernel= np.array(mmd2_lensed_linear_kernel)
# np.save("MMD2_lensed_linear_kernel",mmd2_lensed_linear_kernel)
# mmd2_lensed_linear_kernel = np.load("MMD2_lensed_linear_kernel.npy")

In [ ]:
plt.figure()
plt.plot(kappamap_225[pixels]**2, mmd2_lensed_linear_kernel, '.', label='Lensed (Linear kernel)', alpha=0.3)
plt.plot(kappamap_225[pixels]**2, mmd2_lensed_linear_exakt, '.', label=r'Lensed $(mean_{pixel} - mean_{global})^2$', alpha=0.3)
plt.xlabel(r'$\kappa^2$')
plt.ylabel(r'$MMD^2$(linear kernel)')
plt.legend()
plt.show()

In [ ]:
x = kappamap_225[pixels]**2
y = mmd2_lensed_linear_exakt
# y = mmd2_lensed_linear_kernel

n_bins = 24
bins = np.linspace(x.min(), x.max(), n_bins + 1)

# bin indices in [0, n_bins-1]
idx = np.digitize(x, bins) - 1
idx = np.clip(idx, 0, n_bins - 1)

bin_centers = 0.5*(bins[1:] + bins[:-1])

means = np.full(n_bins, np.nan)
stds  = np.full(n_bins, np.nan)
counts = np.zeros(n_bins, dtype=int)

for i in range(n_bins):
    m = (idx == i)
    c = m.sum()
    counts[i] = c
    # print(f"Bin {i}: count = {c}")
    if c >= 1:
        means[i] = y[m].mean()
    if c >= 2:
        stds[i] = y[m].std(ddof=1)

ses = np.where(counts >= 2, stds / np.sqrt(counts), np.nan)

# keep only well-populated bins
min_count = 5
mask = (counts >= min_count) & np.isfinite(means)
xc, yc, sec = bin_centers[mask], means[mask], ses[mask]

# (optional) weighted fit (weights ~ 1/SE^2)
w = np.where(np.isfinite(sec) & (sec > 0), 1.0/sec**2, 1.0)
fit_params_linear, cov_linear = np.polyfit(xc, yc, 1, w=w, cov=True)   # FRAGE : MIT ODER OHNE GEWICHTUNG?
print(f"MMD² ≈ {fit_params_linear[0]:.3e} * κ² + {fit_params_linear[1]:.3e}")

plt.errorbar(xc, yc, yerr=sec, fmt='o', capsize=3)
plt.plot(bin_centers, fit_params_linear[0]*bin_centers + fit_params_linear[1], 'r--', label='fit')
plt.xlabel(r'$\kappa^2$'); plt.ylabel(r'$\mathrm{MMD}^2$ (linear kernel)')
plt.title('Binned MMD² vs κ² (linspace)')
plt.plot(x,y, '.', alpha=0.1)
plt.legend(); plt.show()


In [ ]:
# Function that plots the binned plots
from scipy.optimize import curve_fit

def plot_binned_MMD2(kappa_squared= True,y = mmd2_lensed_linear_exakt, binning_method = 'lin'):
    # y = mmd2_lensed_linear_exakt
    # y = mmd2_lensed_list
    if kappa_squared:
        x = kappamap_225[pixels]**2
    else:
        x = kappamap_225[pixels]
    
    n_bins = 24
    if binning_method == 'lin':
        bins = np.linspace(x.min(), x.max(), n_bins + 1)
        title_str = 'linspace'
    elif binning_method == 'log':
        bins = np.logspace(np.log10(x.min() + 1e-8), np.log10(x.max()), n_bins + 1)
        title_str = 'logspace'
    elif binning_method == 'quantile':
        bins = np.quantile(x, np.linspace(0, 1, n_bins + 1))
        bins[0] -= 1e-12  # include min
        title_str = 'quantile'
    else:
        raise ValueError("Invalid binning method. Choose 'lin', 'log', or 'quantile'.") 
    
    idx = np.digitize(x, bins) - 1
    idx = np.clip(idx, 0, n_bins - 1)   

    bin_centers = 0.5*(bins[1:] + bins[:-1])
    means = np.full(n_bins, np.nan)
    stds  = np.full(n_bins, np.nan)
    counts = np.zeros(n_bins, dtype=int)
    for i in range(n_bins):
        m = (idx == i)
        c = m.sum()
        counts[i] = c
        if c >= 1:
            means[i] = y[m].mean()
        if c >= 2:
            stds[i] = y[m].std(ddof=1)
        
    ses = np.where(counts >= 2, stds / np.sqrt(counts), np.nan)

    # keep only well-populated bins
    min_count = 5
    mask = (counts >= min_count) & np.isfinite(means)
    xc, yc, sec = bin_centers[mask], means[mask], ses[mask]

    # (optional) weighted fit (weights ~ 1/SE^2)
    w = np.where(np.isfinite(sec) & (sec > 0), 1.0/sec**2, 1.0)
    if kappa_squared:
        title = rf'Binned MMD² vs $\kappa^2$ ({title_str} bins)'
        slope, intercept = np.polyfit(xc, yc, 1, w=w)
        print(f"MMD² ≈ {slope:.3e} * κ² + {intercept:.3e}")
        plt.plot(bin_centers, slope*bin_centers + intercept, 'r--', label='fit')
    else:
        title = rf'Binned MMD² vs $\kappa$ ({title_str} bins)'
        a2, a1, a0 = np.polyfit(xc, yc, 2, w=w)
        print(f"MMD² ≈ {a2:.3e} * κ² + {a1:.3e} * κ + {a0:.3e}")
        plt.plot(bin_centers, a2*bin_centers**2 + a1*bin_centers + a0, 'r--', label='parabola fit')

        #b=0:
        model_even = lambda kappa, a, c: a * (kappa**2) + c

        popt, pcov = curve_fit(model_even, xc[mask], yc[mask])  # honors sigma as true SE
        a, c = popt
        a_err, c_err = np.sqrt(np.diag(pcov))
        print(f"MMD² ≈ {a:.3e} κ² + {c:.3e}  (± {a_err:.1e}, {c_err:.1e})")
        plt.plot(bin_centers, model_even(bin_centers, a, c), 'g--', label=r'$ax^2 + c$ fit')

    plt.errorbar(xc, yc, yerr=sec, fmt='o', capsize=3)
    plt.xlabel(rf'$\kappa^2$'); plt.ylabel(r'$\mathrm{MMD}^2$ (linear kernel)')
    plt.title(title)
    # plt.xlim(0, 0.0001)
    plt.legend(); plt.show()


In [ ]:
plot_binned_MMD2(kappa_squared=True, y=mmd2_lensed_linear_exakt, binning_method='lin')
plot_binned_MMD2(kappa_squared=True, y=mmd2_lensed_linear_kernel, binning_method='lin')

In [ ]:
def plot_binned_MMD_signed(x, y, binning_method = 'lin', n_bins=24, show=True):
    
    if binning_method == 'lin':
        bins = np.linspace(x.min(), x.max(), n_bins + 1)
        title_str = 'linspace'
    elif binning_method == 'log':
        bins = np.logspace(np.log10(x.min() + 1e-8), np.log10(x.max()), n_bins + 1)
        title_str = 'logspace'
    elif binning_method == 'quantile':
        bins = np.quantile(x, np.linspace(0, 1, n_bins + 1))
        bins[0] -= 1e-12  # include min
        title_str = 'quantile'
    else:
        raise ValueError("Invalid binning method. Choose 'lin', 'log', or 'quantile'.")
    
    idx = np.digitize(x, bins) - 1
    idx = np.clip(idx, 0, n_bins - 1)

    bin_centers = 0.5*(bins[1:] + bins[:-1])

    means = np.full(n_bins, np.nan)
    stds= np.full(n_bins, np.nan)
    counts = np.zeros(n_bins, dtype=int)

    for i in range(n_bins):
        m = (idx == i)
        c = m.sum()
        counts[i] = c
        if c >= 1:
            means[i] = y[m].mean()
            
        if c >= 2:
            stds[i] = y[m].std(ddof=1)


    # keep only well-populated bins
    min_count = 5
    mask = counts >= min_count

    ses = np.where(counts >= 2, stds / np.sqrt(counts), np.nan)
    xc, yc, sec = bin_centers[mask], means[mask], ses[mask]


    # Weighted fit (weights ~ 1/SE^2)
    w = np.where(np.isfinite(sec) & (sec > 0), 1.0/sec**2, 1.0)
    slope, intercept = np.polyfit(xc, yc, 1, w=w)
    print(f"MMD ≈ {slope:.3e} * κ + {intercept:.3e}")
    plt.plot(bin_centers, slope*bin_centers + intercept, 'r--', label='Weighted Fit Exakt MMD')

    plt.errorbar(xc, yc, yerr=sec, fmt='o', capsize=3, label='Exakt MMD $(\mu_{pix} - \mu_{glob})$')
    plt.xlabel(r'$\kappa$'); plt.ylabel(r'$\mathrm{MMD}$ (linear kernel)')


    plt.legend()
    title = rf'Binned signed MMD vs $\kappa$ ({title_str} bins)'
    plt.title(title)
    if show:
        plt.show()
    return  xc, yc, sec


### Noise Level with exakt linear Kernel
Get noise level:
We get many MMDs between random samples, take their mean and std and plot it as a horizontal band. If our signal is outside of this band, we know it is a real singal.

In [ ]:
mmd2_noise = []
rng = np.random.default_rng(42)

size = 20000  # match your per-pixel (or subsample) size
for _ in range(15000):
    # Xs = rng.choice(observed_size1000_masked, size=size, replace=False)
    # Ys = rng.choice(observed_size1000_masked, size=size, replace=False)

    Xs = rng.choice(observed_size1000_masked, size=size, replace=True)
    Ys = rng.choice(observed_size1000_masked, size=size, replace=True)
    mmd2_noise.append( (Xs.mean() - Ys.mean())**2 )

mmd2_noise = np.array(mmd2_noise)
noise_mean = mmd2_noise.mean()
noise_std  = mmd2_noise.std()


print("noise mean:", noise_mean, r"$\pm$", noise_std  )

In [ ]:
plt.style.use('ggplot')
from matplotlib.ticker import ScalarFormatter


plt.errorbar(xc, yc, yerr=sec, fmt='.', capsize=3, label=r'Exakt MMD $(\mu_{pix} - \mu_{glob})^2$', color='blue')
plt.plot(bin_centers, fit_params_linear[0]*bin_centers + fit_params_linear[1], 'r--', label=r'Weighted Fit Exakt: MMD$^2$={:.2e}$\cdot\kappa^2$ + {:.2e}'.format(fit_params_linear[0], fit_params_linear[1]))
plt.axhline(noise_mean, color='black', linestyle='--', label='Noise Mean')
plt.fill_between([0,0.0007], 
                 noise_mean - noise_std, 
                 noise_mean + noise_std,
                 color='black', alpha=0.5, label='Noise ±1σ')
plt.xlim(0, 0.0004)
plt.legend()
plt.xlabel(r'$\kappa^2$'); plt.ylabel(r'$\mathrm{MMD}^2$ (linear kernel)')
plt.title('Binned MMD² vs κ² (linspace)')

ax = plt.gca()
fmt = ScalarFormatter(useMathText=True)
fmt.set_powerlimits((0, 0))         # always use scientific notation
ax.xaxis.set_major_formatter(fmt)
ax.ticklabel_format(axis='x', style='sci', scilimits=(0, 0), useMathText=False)
ax.ticklabel_format(useOffset=True) # keep the scientific scale as an offset text
plt.plot(x,y, '.', alpha=0.1)
plt.ylim(-0.000005, 0.00015)
plt.tight_layout()
plt.savefig('images/MMD2_vs_kappa2_linear.jpg', dpi=300)
plt.show()

slope, intercept = fit_params_linear
slope_err, intercept_err = np.sqrt(np.diag(cov_linear))
print(f"Final fit result: MMD² ≈ ({slope:.3e} ± {slope_err:.1e}) * κ² + ({intercept:.3e} ± {intercept_err:.1e})")

yc_preds = slope*xc + intercept
r2 = r2_score(yc, yc_preds)
print(f"R² of the fit: {r2:.4f}")

In [ ]:
# Determine the kappa² limit where MMD² rises above noise + 1σ

kappa2_noise_threshold = np.linspace(0.00001, 0.00005, 50)
mmd2_noise_threshold = fit_params_linear[0]*kappa2_noise_threshold + fit_params_linear[1]

above_noise = mmd2_noise_threshold > (noise_mean + noise_std)
kappa2_limit = kappa2_noise_threshold[above_noise][0]

print(f"Kappa² limit where MMD² rises above noise + 1σ: {kappa2_limit:.2e}")

### Finer Convergence map
Now we want to apply a finer/smoother convergence map to the galaxies. We will then compute the MMD of the coarser pixels, such that each bigger pixel represents a neighbourhood of galaxies with slightly different kappas.

In [ ]:
# Create a smoother convergence map.
# From now on, we only use the big catalogue
nside_fine = 1024

kappamap_fine = hp.synfast( cl_kappa_225, nside_fine)
print(hp.nside2npix(nside_fine))

In [ ]:
# Convert galaxy coordinates to pixel numbers in the finer map
gal_pix_fine = hp.ang2pix(nside_fine, catalogue1000['ra'], catalogue1000['dec'], lonlat=True)
gal_pix_fine_unique, gal_pix_fine_counts = np.unique(gal_pix_fine, return_counts=True)

print("Galaxy pixels:",gal_pix_fine, "   Length of Galaxy pixels(should match nr. of galaxies):", len(gal_pix_fine))
print("Number of unique pixels with galaxies:", len(gal_pix_fine_unique))
print("Max number of galaxies in a pixel:", np.max(gal_pix_fine_counts))
print("Min number of galaxies in a pixel:", np.min(gal_pix_fine_counts))
print("Mean number of galaxies in a pixel:", np.mean(gal_pix_fine_counts))
print("Number of pixels with > 20'000 galaxies:", np.sum(gal_pix_fine_counts >= 20000))
print('-'*30)


# Compute observed sizes for all galaxies using the finer kappa map
observed_size_fine = intrinsic_size1000 * (1.0 + kappamap_fine[gal_pix_fine])


In [ ]:
mmd2_lensed_rbf_fine_batch1 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_1.npy")
mmd2_lensed_rbf_fine_batch2 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_2.npy")
mmd2_lensed_rbf_fine_batch3 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_3.npy")
mmd2_lensed_rbf_fine_batch4 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_4.npy")
mmd2_lensed_rbf_fine_batch5 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_5.npy")
mmd2_lensed_rbf_fine_batch6 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_6.npy")
mmd2_lensed_rbf_fine_batch7 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_7.npy")
mmd2_lensed_rbf_fine_batch8 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_8.npy")
mmd2_lensed_rbf_fine_batch9 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_9.npy")
mmd2_lensed_rbf_fine_batch10 = np.load(f"mmd2_lensed_rbf_fine/mmd2_lensed_rbf_fine_batch_10.npy")

In [ ]:
kappamap_avg_fine = np.load('kappa_avg_fine.npy')
mmd2_rbf_noise_fine = np.load('mmd2_rbf_noise.npy')
mmd2_rbf_noise_mean_fine = np.mean(mmd2_rbf_noise_fine)
mmd2_rbf_noise_std_fine = np.std(mmd2_rbf_noise_fine, ddof=1)

mmd2_lensed_rbf_fine = np.concatenate((mmd2_lensed_rbf_fine_batch1, mmd2_lensed_rbf_fine_batch2,
                                       mmd2_lensed_rbf_fine_batch3, mmd2_lensed_rbf_fine_batch4,
                                       mmd2_lensed_rbf_fine_batch5, mmd2_lensed_rbf_fine_batch6,
                                       mmd2_lensed_rbf_fine_batch7, mmd2_lensed_rbf_fine_batch8,
                                       mmd2_lensed_rbf_fine_batch9, mmd2_lensed_rbf_fine_batch10), axis=0)

mask_20k_gals = galaxy_pix1000_counts >= 20000  # pixels with at least 20k galaxies
kappamap_avg_fine = kappamap_avg_fine[mask_20k_gals]


In [ ]:
n_bins = 24
bins = np.linspace(np.min(kappamap_avg_fine**2), np.max(kappamap_avg_fine**2), n_bins + 1)

# bin indices in [0, n_bins-1]
idx = np.digitize(kappamap_avg_fine**2, bins) - 1
idx = np.clip(idx, 0, n_bins - 1)

bin_centers = 0.5*(bins[1:] + bins[:-1])

means = np.full(n_bins, np.nan)
stds  = np.full(n_bins, np.nan)
counts = np.zeros(n_bins, dtype=int)

for i in range(n_bins):
    m = (idx == i)
    c = m.sum()
    counts[i] = c
    # print(f"Bin {i}: count = {c}")
    if c >= 1:
        means[i] = mmd2_lensed_rbf_fine[m].mean()
    if c >= 2:
        stds[i] = mmd2_lensed_rbf_fine[m].std(ddof=1)

print("COUNTS:", counts)
ses = np.where(counts >= 2, stds / np.sqrt(counts), np.nan)

# keep only well-populated bins
min_count = 5
mask = (counts >= min_count) & np.isfinite(means)
xc, yc, sec = bin_centers[mask], means[mask], ses[mask]

# (optional) weighted fit (weights ~ 1/SE^2)
w = np.where(np.isfinite(sec) & (sec > 0), 1.0/sec**2, 1.0)
slope, intercept = np.polyfit(xc, yc, 1, w=w)   # FRAGE : MIT ODER OHNE GEWICHTUNG?
print(f"MMD² ≈ {slope:.3e} * κ² + {intercept:.3e}")

plt.plot(kappamap_avg_fine**2, mmd2_lensed_rbf_fine, '.', label='Lensed (MMD RBF)', alpha=0.2)

plt.errorbar(xc, yc, yerr=sec, fmt='o', capsize=3)
plt.plot(bin_centers, slope*bin_centers + intercept, 'r--', label='Weighted Fit')
plt.xlabel(r'$\bar{\kappa}^2$'); plt.ylabel(r'$\mathrm{MMD}^2$ (RBF kernel)')
plt.axhline(mmd2_rbf_noise_mean_fine, color='black', linestyle='--', label='Noise Mean')
plt.fill_between([0,0.0007], 
                 mmd2_rbf_noise_mean_fine - mmd2_rbf_noise_std_fine, 
                 mmd2_rbf_noise_mean_fine + mmd2_rbf_noise_std_fine,
                 color='black', alpha=0.5, label='Noise ±1σ')
plt.title(r'Binned MMD² vs $\bar{\kappa}²$ (linspace)')
# plt.ylim(0.0001, 0.000125)
plt.xlim(np.min(bins)-1e-5, np.max(bins))
plt.legend(); plt.show()

print(xc)
print(yc)
print(mmd2_rbf_noise_mean_fine + mmd2_rbf_noise_std_fine)

In [ ]:
means_fine = []
for pix in galaxy_pix1000_unique_masked:
    mask = (galaxy_pix1000 == pix)
    if sum(mask) > 20000:
        means_fine.append(np.mean(observed_size_fine[mask]))

means_fine = np.array(means_fine)
plus_min_mask_fine = np.sign(means_fine - intrinsic_size1000_masked.mean())


In [ ]:
sign_fine = np.sign(means_fine - observed_size1000_masked.mean())
print(sign_fine)
plus_min_mask_fine = sign_fine > 0
print(plus_min_mask_fine)


In [ ]:
from sklearn.metrics import r2_score
def binning_and_plotting(x,y, n_bins = 24, Title: str = None, plot=True):
    bins = np.linspace(x.min(), x.max(), n_bins + 1)

    # bin indices in [0, n_bins-1]
    idx = np.digitize(x, bins) - 1
    idx = np.clip(idx, 0, n_bins - 1)
    bin_centers = 0.5*(bins[1:] + bins[:-1])

    means = np.full(n_bins, np.nan)
    stds  = np.full(n_bins, np.nan)
    counts = np.zeros(n_bins, dtype=int)
    for i in range(n_bins):
        m = (idx == i)
        c = m.sum()
        counts[i] = c
        # print(f"Bin {i}: count = {c}")
        if c >= 1:
            means[i] = y[m].mean()
        if c >= 2:
            stds[i] = y[m].std(ddof=1)


    # keep only well-populated bins
    min_count = 5
    mask = counts >= min_count

    ses = np.where(counts >= 2, stds / np.sqrt(counts), np.nan)
    xc, yc, sec = bin_centers[mask], means[mask], ses[mask]

    # (optional) weighted fit (weights ~ 1/SE^2)
    w = np.where(np.isfinite(sec) & (sec > 0), 1.0/sec**2, 1.0)
    fit_params, cov = np.polyfit(xc, yc, 1, w=w, cov=True)
    print(f"MMD ≈ {fit_params[0]:.3e} * κ + {fit_params[1]:.3e}")
    if not plot:
        return xc, yc, sec, fit_params
    else:
        # plt.figure()
        plt.plot(x,y, '.', label=r'MMD$^2$ per Pixel', alpha=0.2)
        plt.errorbar(xc, yc, yerr=sec, fmt='o', capsize=3)
        # plt.plot(bin_centers, fit_params[0]*bin_centers + fit_params[1], 'r--', label='fit')
        # plt.xlabel(r'$\kappa$'); plt.ylabel(r'$\mathrm{MMD}$ (RBF kernel)')
        plt.title(Title if not Title is None else ' ')
        
        # plt.legend(); plt.show()

    return  xc, yc, sec, fit_params, cov


In [ ]:
# sign_fine = plus_min_mask_fine
plus_min_mask_fine = sign_fine < 0


### Recovering the Convergence Map and Power Spectrum
We create a new convergence map. Apply it to the intrinsic galaxy sizes and compute the MMD just as before. We compute the MMD with rbf and custom kernel. Then use the estimators we obtained with the earlier kappamap_fine to try to recover the kappa values of the new convergence map.

#### RBF Kernel
We recover the convergence map (seed=31) from the MMD measurements with the RBFkernel

In [ ]:
# Create and apply new convergence map for RECOVERY test
recovery_rand_seed = 31
np.random.seed(recovery_rand_seed)
recovery_kappamap = hp.synfast(cl_kappa_225, nside_fine)
recovery_observed_size = intrinsic_size1000 * (1.0 + recovery_kappamap[gal_pix_fine])



In [ ]:
means_recov_fine = []
for pix in galaxy_pix1000_unique:
    mask = (galaxy_pix1000 == pix)
    if sum(mask) > 20000:
        means_recov_fine.append(np.mean(recovery_observed_size[mask]))

means_recov_fine = np.array(means_recov_fine)
sign_recov_fine = np.sign(means_recov_fine - recovery_observed_size.mean())

In [ ]:
mmd2_recov_rbf_batch1 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_1.npy")
mmd2_recov_rbf_batch2 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_2.npy")
mmd2_recov_rbf_batch3 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_3.npy")
mmd2_recov_rbf_batch4 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_4.npy")
mmd2_recov_rbf_batch5 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_5.npy")
mmd2_recov_rbf_batch6 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_6.npy")
mmd2_recov_rbf_batch7 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_7.npy")
mmd2_recov_rbf_batch8 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_8.npy")
mmd2_recov_rbf_batch9 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_9.npy")
mmd2_recov_rbf_batch10 = np.load(f"mmd2_recov_rbf/mmd2_lensed_rbf_fine_batch_10.npy")

mmd2_recov_rbf = np.concatenate((mmd2_recov_rbf_batch1, mmd2_recov_rbf_batch2,
                                 mmd2_recov_rbf_batch3, mmd2_recov_rbf_batch4,
                                 mmd2_recov_rbf_batch5, mmd2_recov_rbf_batch6,
                                 mmd2_recov_rbf_batch7, mmd2_recov_rbf_batch8,
                                 mmd2_recov_rbf_batch9, mmd2_recov_rbf_batch10), axis=0)
kappa_recov_avg_fine = np.load('kappa_recov_avg_fine.npy')
kappa_recov_avg_fine = kappa_recov_avg_fine[mask_20k_gals]      # Want to predict these kappa values

signed_kappa2_recov = np.sign(kappa_recov_avg_fine) * kappa_recov_avg_fine**2   # Correct sign assignments: 887/1145
signed_mmd2_recov_rbf = sign_recov_fine * mmd2_recov_rbf

In [ ]:
recov_xc, recov_yc, recov_sec, recov_fit, cov_fit = binning_and_plotting(signed_kappa2_recov, signed_mmd2_recov_rbf, n_bins=24, Title=r'Binned MMD² vs $sgn(\kappa)\cdot\kappa²$ (Recovery Map)', plot=True)
plt.plot(np.linspace(-0.0004, 0.0004, 10), recov_fit[0]*np.linspace(-0.0004, 0.0004, 10) + recov_fit[1], linestyle='--', color='black', label=r'Weighted fit: MMD={:.2e}$\cdot\kappa$ + {:.2e}'.format(recov_fit[0], recov_fit[1]))

plt.xlabel(r'$sgn(\kappa)\cdot\kappa²$')
plt.ylabel(r'$MMD²$ (RBF kernel)')
plt.title(r'Binned MMD² vs $sgn(\kappa)\cdot\kappa^2$ (RBF)')
plt.legend()
plt.tight_layout()
plt.savefig('images/recovery_mmd2_vs_kappa2_rbf.jpg', dpi=300)
plt.show()

slope, intercept = recov_fit
slope_unc, intercept_unc = np.sqrt(np.diag(cov_fit))

print(f"Slope: {slope} ± {slope_unc}")
print(f"Intercept: {intercept} ± {intercept_unc}")

recov_yc_preds = slope * recov_xc + intercept
r2_recov = r2_score(recov_yc, recov_yc_preds)
print(f"R² score for recovery fit: {r2_recov:.4f}"  )

In [ ]:
pred_kappa_recov = np.empty_like(kappa_recov_avg_fine)  # Thats the vector we want to fill with predicted kappa values


pred_kappa2_recov_pos = signed_mmd2_recov_rbf[signed_mmd2_recov_rbf > 0] / estimator_pos[0] - estimator_pos[1]/estimator_pos[0]     # Problem: not all values are positive because of wrong sign assignment
pred_kappa2_recov_neg = signed_mmd2_recov_rbf[signed_mmd2_recov_rbf < 0] / estimator_neg[0] - estimator_neg[1]/estimator_neg[0]

pred_kappa_recov[signed_mmd2_recov_rbf > 0] = np.sign(pred_kappa2_recov_pos) * np.sqrt(np.abs(pred_kappa2_recov_pos))
# pred_kappa_recov[signed_mmd2_recov_rbf < 0] = 

In [ ]:
(np.sign(pred_kappa2_recov_pos) == np.sign(signed_mmd2_recov_rbf[signed_mmd2_recov_rbf>0])).sum()

In [ ]:
print((-1==np.sign(pred_kappa2_recov_neg)).sum())
print(np.sign(pred_kappa2_recov_neg).shape)

In [ ]:
plt.plot(pred_kappa2_recov_pos, signed_kappa2_recov[signed_mmd2_recov_rbf > 0], '.', alpha=0.2, label='Recovered Positive MMD')

#### Directed RBF Kernel
Now we do the same as above, but with the MMDs measured with the custom directed RBF kernel.


In [ ]:
mmd2_recov_dir_rbf_batch1 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_1.npy")
mmd2_recov_dir_rbf_batch2 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_2.npy")
mmd2_recov_dir_rbf_batch3 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_3.npy")
mmd2_recov_dir_rbf_batch4 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_4.npy")
mmd2_recov_dir_rbf_batch5 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_5.npy")
mmd2_recov_dir_rbf_batch6 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_6.npy")
mmd2_recov_dir_rbf_batch7 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_7.npy")
mmd2_recov_dir_rbf_batch8 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_8.npy")
mmd2_recov_dir_rbf_batch9 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_9.npy")
mmd2_recov_dir_rbf_batch10 = np.load(f"mmd2_recov_dir_rbf/mmd2_lensed_dir_rbf_fine_batch_10.npy")

mmd2_recov_dir_rbf_fine = np.concatenate((mmd2_recov_dir_rbf_batch1, mmd2_recov_dir_rbf_batch2,
                                           mmd2_recov_dir_rbf_batch3, mmd2_recov_dir_rbf_batch4,
                                           mmd2_recov_dir_rbf_batch5, mmd2_recov_dir_rbf_batch6,
                                           mmd2_recov_dir_rbf_batch7, mmd2_recov_dir_rbf_batch8,
                                           mmd2_recov_dir_rbf_batch9, mmd2_recov_dir_rbf_batch10), axis=0)
mmd2_recov_dir_rbf_fine = (-1) * mmd2_recov_dir_rbf_fine  # Correct sign convention


In [ ]:
recov_dir_xc, recov_dir_yc, recov_dir_sec, recov_dir_fit, cov_dir_fit = binning_and_plotting(signed_kappa2_recov, mmd2_recov_dir_rbf_fine, n_bins=24, Title=r'Binned MMD² vs $sgn(\kappa)\cdot\kappa²$ (Recovery Map, Directional RBF)', plot=True)
plt.plot(np.linspace(-0.0004, 0.0004, 10), recov_dir_fit[0]*np.linspace(-0.0004, 0.0004, 10) + recov_dir_fit[1], linestyle='--', color='black', label=r'Weighted fit: MMD^2={:.2e}$\cdot\kappa^2$ + {:.2e}'.format(recov_dir_fit[0], recov_dir_fit[1]))
plt.xlabel(r'$sgn(\kappa)\cdot\kappa²$')
plt.ylabel(r'$MMD²$ (Dir. RBF kernel)')
plt.title('Binned MMD² vs $sgn(\kappa)\cdot\kappa²$ (Directional RBF)')
plt.legend()
plt.tight_layout()
plt.savefig("images/recovery_mmd2_vs_kappa2_directional_rbf.jpg", dpi=300)
plt.show()

slope, intercept = recov_dir_fit
slope_unc, intercept_unc = np.sqrt(np.diag(cov_dir_fit))

print(f"Slope: {slope} ± {slope_unc}")
print(f"Intercept: {intercept} ± {intercept_unc}")


recov_dir_yc_preds = slope * recov_dir_xc + intercept
r2_recov_dir = r2_score(recov_dir_yc, recov_dir_yc_preds)
print(f"R² score for recovery fit: {r2_recov_dir:.4f}"  )

### Create a new estimator
What we will do here:
- On euler, we will create a new fine convergence map kappa_est_fine. (set random seed)!
- We will apply this convergence to the intrinsic sizes
- compute the MMD^2 between coarse pixel and global distribution
- compute the average kappa in each coarse pixel

- Upload here the kappa_avg_est and MMD2_rbf_est
- compute mean-shift
- Plot as above sgn(kappa)*kappa^2 vs +/- MMD2 (the sign of the MMD is given by mean-shift)
- Fit the binned values and get an estimator.


#### Directed RBF kernel

In [ ]:
mmd2_dir_rbf_fine_batch1 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_1.npy")
mmd2_dir_rbf_fine_batch2 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_2.npy")
mmd2_dir_rbf_fine_batch3 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_3.npy")
mmd2_dir_rbf_fine_batch4 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_4.npy")
mmd2_dir_rbf_fine_batch5 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_5.npy")
mmd2_dir_rbf_fine_batch6 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_6.npy")
mmd2_dir_rbf_fine_batch7 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_7.npy")
mmd2_dir_rbf_fine_batch8 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_8.npy")
mmd2_dir_rbf_fine_batch9 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_9.npy")
mmd2_dir_rbf_fine_batch10 = np.load(f"mmd2_dir_rbf_fine3/mmd2_dir_rbf_fine_batch_10.npy")

mmd2_dir_rbf_fine = np.concatenate((mmd2_dir_rbf_fine_batch1, mmd2_dir_rbf_fine_batch2,
                                    mmd2_dir_rbf_fine_batch3, mmd2_dir_rbf_fine_batch4,
                                    mmd2_dir_rbf_fine_batch5, mmd2_dir_rbf_fine_batch6,
                                    mmd2_dir_rbf_fine_batch7, mmd2_dir_rbf_fine_batch8,
                                    mmd2_dir_rbf_fine_batch9, mmd2_dir_rbf_fine_batch10), axis=0)
kappa_avg_fine4 = np.load('kappa_avg_fine4.npy')
kappa_avg_fine4 = kappa_avg_fine4[mask_20k_gals]

signed_kappa2_avg_fine4 = np.sign(kappa_avg_fine4) * kappa_avg_fine4**2
mmd2_dir_rbf_fine = (-1) * mmd2_dir_rbf_fine  # Correct sign convention

In [ ]:
((np.sign(mmd2_dir_rbf_fine)== np.sign(signed_kappa2_avg_fine4)).sum())/len(mmd2_dir_rbf_fine)

In [ ]:
fig2 = plt.figure()
fig2.patch.set_facecolor(bg_color)
est_dir_xc, est_dir_yc, est_sec, est_dir_fit, est_cov_fit = binning_and_plotting(signed_kappa2_avg_fine4, mmd2_dir_rbf_fine, n_bins=24, Title=r'Binned MMD$^2$ vs $sgn(\kappa)\cdot\kappa^2$ (Directed RBF)', plot=True)
plt.plot(np.linspace(-0.0004, 0.0004, 10), est_dir_fit[0]*np.linspace(-0.0004, 0.0004, 10) + est_dir_fit[1], linestyle='--', color='black', label='Weighted fit')
# plt.plot(np.linspace(-0.0004, 0.0004, 10), a*np.linspace(-0.0004, 0.0004, 10) + b, linestyle='--', color='Green', label=r'Weighted fit: MMD^2={:.2e}$\cdot\kappa^2$ + {:.2e}'.format(a, b))
plt.xlabel(r'$sgn(\kappa)\cdot\kappa^2$', fontsize=18)
plt.ylabel(r'$MMD^2$', fontsize=18)
plt.title(r'Directed RBF', fontsize=20)
plt.legend(loc='upper left', fontsize=12)
plt.tight_layout()

# plt.savefig("images/estimator_mmd2_vs_kappa2_dir_rbf.jpg", dpi=300)
plt.show()

slope, intercept = est_dir_fit
slope_unc, intercept_unc = np.sqrt(np.diag(est_cov_fit))

print(f"Slope: {slope} ± {slope_unc}")
print(f"Intercept: {intercept} ± {intercept_unc}")
est_dir_yc_preds = slope * est_dir_xc + intercept
r2_est_dir = r2_score(est_dir_yc, est_dir_yc_preds)
print(f"R² score for estimation fit: {r2_est_dir:.4f}"  )

In [ ]:
a, b = np.polyfit(signed_kappa2_avg_fine4, mmd2_dir_rbf_fine, 1)
print(f"MMD² ≈ {a:.3e} * κ² + {b:.3e}")

#### RBF Kernel

In [ ]:
estimator_seed = 19
np.random.seed(estimator_seed)
estimator_kappamap = hp.synfast(cl_kappa_225, nside_fine)
estimator_observed_size = intrinsic_size1000 * (1.0 + estimator_kappamap[gal_pix_fine])

In [ ]:
means_est_fine = []
for pix in galaxy_pix1000_unique:
    mask = (galaxy_pix1000 == pix)
    if sum(mask) > 20000:
        means_est_fine.append(np.mean(estimator_observed_size[mask]))

means_est_fine = np.array(means_est_fine)
sign_est_fine = np.sign(means_est_fine - estimator_observed_size.mean())

In [ ]:
mmd2_est_rbf_fine_batch1 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_1.npy")
mmd2_est_rbf_fine_batch2 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_2.npy")
mmd2_est_rbf_fine_batch3 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_3.npy")
mmd2_est_rbf_fine_batch4 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_4.npy")
mmd2_est_rbf_fine_batch5 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_5.npy")
mmd2_est_rbf_fine_batch6 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_6.npy")
mmd2_est_rbf_fine_batch7 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_7.npy")
mmd2_est_rbf_fine_batch8 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_8.npy")
mmd2_est_rbf_fine_batch9 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_9.npy")
mmd2_est_rbf_fine_batch10 = np.load(f"mmd2_est_rbf_fine2/mmd2_est_rbf_fine_batch_10.npy")

mmd2_est_rbf_fine = np.concatenate((mmd2_est_rbf_fine_batch1, mmd2_est_rbf_fine_batch2,
                                    mmd2_est_rbf_fine_batch3, mmd2_est_rbf_fine_batch4,
                                    mmd2_est_rbf_fine_batch5, mmd2_est_rbf_fine_batch6,
                                    mmd2_est_rbf_fine_batch7, mmd2_est_rbf_fine_batch8,
                                    mmd2_est_rbf_fine_batch9, mmd2_est_rbf_fine_batch10), axis=0)

kappa_est_avg_fine3 = np.load('kappa_avg_fine4.npy')
kappa_est_avg_fine3 = kappa_est_avg_fine3[mask_20k_gals]

signed_kappa2_est_fine3 = np.sign(kappa_est_avg_fine3) * kappa_est_avg_fine3**2
signed_mmd2_est_rbf = sign_est_fine * mmd2_est_rbf_fine

In [ ]:
((sign_est_fine == np.sign(kappa_est_avg_fine3)).sum())/len(sign_est_fine)

In [ ]:
fig = plt.figure()
est_rbf_xc, est_rbf_yc, est_rbf_sec, est_rbf_fit, est_rbf_cov_fit = binning_and_plotting(signed_kappa2_est_fine3, signed_mmd2_est_rbf, n_bins=24, Title=r'Binned MMD² vs $sgn(\kappa)\cdot\kappa^2$ (Estimation Map, RBF)', plot=True)
plt.plot(np.linspace(-0.0004, 0.0004, 10), est_rbf_fit[0]*np.linspace(-0.0004, 0.0004, 10) + est_rbf_fit[1], linestyle='--', color='black', label='Weighted fit')
plt.xlabel(r'$sgn(\kappa)\cdot\kappa^2$', fontsize=18)
plt.ylabel(r'$MMD^2$', fontsize=18)
plt.title(r'RBF Kernel', fontsize=20)
plt.tight_layout()
# plt.xlim(-0.0006, 0.00045)
# plt.savefig("images/estimator_mmd2_vs_kappa2_rbf.jpg", dpi=300)
# plt.plot(np.linspace(-0.0004, 0.0004, 10), a2*np.linspace(-0.0004, 0.0004, 10) + b2, linestyle='--', color='Green', label=r'Weighted fit: MMD^2={:.2e}$\cdot\kappa^2$ + {:.2e}'.format(a2, b2))
plt.legend(loc='upper left', fontsize=12)
bg_color = '#F7F7F7'
fig.patch.set_facecolor(bg_color)
plt.show()

slope, intercept = est_rbf_fit
slope_unc, intercept_unc = np.sqrt(np.diag(est_rbf_cov_fit))

print(f"Slope: {slope} ± {slope_unc}")
print(f"Intercept: {intercept} ± {intercept_unc}")
est_rbf_yc_preds = slope * est_rbf_xc + intercept
r2_est_rbf = r2_score(est_rbf_yc, est_rbf_yc_preds)
print(f"R² score for estimation fit: {r2_est_rbf:.4f}"  )


In [ ]:
a2, b2 = np.polyfit(signed_kappa2_est_fine3, signed_mmd2_est_rbf, 1)
print(f"MMD² ≈ {a2:.3e} * κ² + {b2:.3e}")

### Actual Recovery of the Convergence Map

#### Pixelwise recovery


In [ ]:
pred_kappa2_dir_rbf = mmd2_recov_dir_rbf_fine/est_dir_fit[0] - est_dir_fit[1]/est_dir_fit[0]
pred_kappa_dir_rbf = np.sign(pred_kappa2_dir_rbf) * np.sqrt(np.abs(pred_kappa2_dir_rbf))

pred_kappa2_rbf = signed_mmd2_recov_rbf/est_rbf_fit[0] - est_rbf_fit[1]/est_rbf_fit[0]
pred_kappa_rbf = np.sign(pred_kappa2_rbf) * np.sqrt(np.abs(pred_kappa2_rbf))



In [ ]:
import scipy.stats as stats
def plot_pixel_level_results(kappa_true, kappa_pred):
    """Generate all pixel-level validation figures"""
    
    mask = ~(np.isnan(kappa_true) | np.isnan(kappa_pred))
    kt, kp = np.asarray(kappa_true)[mask], np.asarray(kappa_pred)[mask]
    residuals = kp - kt
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    fig.suptitle('Pixel-Level Results (Dir. RBF)', fontsize=16)
    # 1. Scatter plot
    ax1 = axes[0, 0]
    ax1.scatter(kt, kp, alpha=0.3, s=5)
    lims = [min(kt.min(), kp.min()), max(kt.max(), kp.max())]
    ax1.plot(lims, lims, 'r--', lw=2)
    ax1.set_xlabel(r'$\kappa_{\rm true}$')
    ax1.set_ylabel(r'$\kappa_{\rm pred}$')
    ax1.set_title(f'r = {np.corrcoef(kt, kp)[0,1]:.3f}')
    
    # 2. Residual histogram
    ax2 = axes[0, 1]
    ax2.hist(residuals, bins=50, density=True, alpha=0.7, edgecolor='black')
    x = np.linspace(residuals.min(), residuals.max(), 200)
    ax2.plot(x, stats.norm.pdf(x, np.mean(residuals), np.std(residuals)), 'r-', lw=2)
    ax2.axvline(0, color='k', linestyle='--')
    ax2.set_xlabel(r'$\kappa_{\rm pred} - \kappa_{\rm true}$')
    ax2.set_ylabel('Density')
    ax2.set_title(f'Mean = {np.mean(residuals):.2e}, Std = {np.std(residuals):.4f}')
    
    # 3. Residuals vs true kappa
    ax3 = axes[1, 0]
    ax3.scatter(kt, residuals, alpha=0.3, s=5)
    ax3.axhline(0, color='r', linestyle='--')
    ax3.set_xlabel(r'$\kappa_{\rm true}$')
    ax3.set_ylabel(r'$\kappa_{\rm pred} - \kappa_{\rm true}$')
    ax3.set_title('Residuals vs true convergence')
    print("Residuals > 0:", (residuals > 0).sum(), "Residuals < 0:", (residuals < 0).sum())
    print("Residuals vs kapp correlation coeff. :", np.corrcoef(kt, residuals)[0,1])
    # 4. Summary statistics text
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    rmse = np.sqrt(np.mean(residuals**2))
    r2 = 1 - np.sum(residuals**2) / np.sum((kt - np.mean(kt))**2)
    nrmse = rmse / np.std(kt)
    
    stats_text = f"""
    Pixel-Level Summary Statistics
    {'='*35}
    
    N pixels:           {len(kt)}
    
    Pearson r:          {np.corrcoef(kt, kp)[0,1]:.4f}
    R²:                 {r2:.4f}
    
    RMSE:               {rmse:.6f}
    NRMSE (σ):          {nrmse:.4f}
    
    Mean residual:      {np.mean(residuals):.2e}
    Std residual:       {np.std(residuals):.6f}
    """
    ax4.text(0.1, 0.5, stats_text, fontsize=14, family='monospace',
             verticalalignment='center', transform=ax4.transAxes)
    
    plt.tight_layout()
    return ax1

In [ ]:
plot_pixel_level_results(kappa_recov_avg_fine, pred_kappa_rbf)
# plt.savefig("images/pixel_level_results_rbf.jpg", dpi=300)
plot_pixel_level_results(kappa_recov_avg_fine, pred_kappa_dir_rbf)
# plt.savefig("images/pixel_level_results_dir_rbf.jpg", dpi=300)

In [ ]:
def plot_residual_map(kappa_true, kappa_pred, galaxy_pix_unique, nside=64, 
                      save_path='residual_map.pdf'):
    """
    Create a HEALPix map showing spatial distribution of residuals.
    
    Parameters:
        kappa_true: array of true convergence values per pixel
        kappa_pred: array of predicted convergence values per pixel
        galaxy_pix_unique: array of pixel indices corresponding to kappa values
        nside: HEALPix nside parameter
        save_path: path to save the figure
    """
    
    # Compute residuals
    residuals = kappa_pred - kappa_true
    
    # Create empty HEALPix map (filled with NaN for empty pixels)
    npix = hp.nside2npix(nside)
    residual_map = np.full(npix, np.nan)
    
    # Fill in the residuals for pixels with data
    residual_map[galaxy_pix_unique] = residuals
    
    # Find the center of your observed patch
    # Get RA, Dec of pixels with data
    theta, phi = hp.pix2ang(nside, galaxy_pix_unique)
    ra = np.degrees(phi)
    dec = 90 - np.degrees(theta)
    
    # Center coordinates (mean of your data)
    ra_center = np.mean(ra)
    dec_center = np.mean(dec)
    
    # Determine color scale limits (symmetric around zero)
    vmax = np.nanmax(np.abs(residuals))
    vmin = -vmax
    
    # Create the plot - use cartview for rectangular projection
    fig = plt.figure(figsize=(12, 8))
    
    hp.cartview(residual_map, 
                title=r'Residual Map: $\kappa_{\rm pred} - \kappa_{\rm true}$',
                cmap='RdBu_r',
                min=vmin,
                max=vmax,
                lonra=[ra_center - 20, ra_center + 20],  # ±20 degrees
                latra=[dec_center - 20, dec_center + 20],
                cbar=True,
                unit=r'$\Delta\kappa$',
                hold=True)
    
    # hp.graticule()
    
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"Residual map saved to {save_path}")
    
    return fig, residual_map


def plot_kappa_comparison_maps(kappa_true, kappa_pred, galaxy_pix_unique, 
                               nside=64, save_path='kappa_comparison.pdf'):
    """
    Create three side-by-side HEALPix maps: true, predicted, and residuals.
    """
    
    npix = hp.nside2npix(nside)
    
    # Create maps
    true_map = np.full(npix, np.nan)
    pred_map = np.full(npix, np.nan)
    residual_map = np.full(npix, np.nan)
    
    true_map[galaxy_pix_unique] = kappa_true
    pred_map[galaxy_pix_unique] = kappa_pred
    residual_map[galaxy_pix_unique] = kappa_pred - kappa_true
    
    # Find center of observed patch
    theta, phi = hp.pix2ang(nside, galaxy_pix_unique)
    ra = np.degrees(phi)
    dec = 90 - np.degrees(theta)
    ra_center = 0 #np.mean(ra)
    dec_center = 0 #np.mean(dec)
    
    # Determine color scales
    vmax_kappa = np.abs(kappa_true).max()       #max(np.nanmax(np.abs(kappa_true)), np.nanmax(np.abs(kappa_pred)))
    vmin_kappa = -vmax_kappa
    
    vmax_res =  np.nanmax(np.abs(kappa_pred - kappa_true))
    vmin_res = -vmax_res
    
    fig = plt.figure(figsize=(18, 5))
    fig.patch.set_facecolor(bg_color)
    # True convergence
    plt.subplot(131)
    hp.cartview(true_map, 
                # title=r'$\kappa_{\rm true}$',
                cmap='RdBu_r',
                min=vmin_kappa,
                max=vmax_kappa,
                lonra=[-20, 20],
                latra=[-20, 20],
                hold=True,
                sub=131)
    plt.title(r'$\kappa_{\rm true}$', fontsize=24)
    # hp.graticule()
    
    # Predicted convergence
    plt.subplot(132)
    hp.cartview(pred_map, 
                # title=r'$\kappa_{\rm pred}$',
                cmap='RdBu_r',
                min=vmin_kappa,
                max=vmax_kappa,
                lonra=[ra_center - 20, ra_center + 20],
                latra=[dec_center - 20, dec_center + 20],
                hold=True,
                sub=132)
    plt.title(r'$\kappa_{\rm pred}$', fontsize=24)
    # hp.graticule()
    
    # Residuals
    plt.subplot(133)
    hp.cartview(residual_map, 
                # title=r'$\kappa_{\rm pred} - \kappa_{\rm true}$',
                cmap='RdBu_r',
                min=vmin_res,
                max=vmax_res,
                lonra=[ra_center - 20, ra_center + 20],
                latra=[dec_center - 20, dec_center + 20],
                hold=True,
                sub=133)
    plt.title(r'$\kappa_{\rm pred} - \kappa_{\rm true}$', fontsize=24)
    # hp.graticule()
    
    # plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"Comparison map saved to {save_path}")
    
    return #fig

In [ ]:
# Set default sizes for all plots
# plt.rcParams['axes.titlesize'] = 24
plt.rcParams['xtick.labelsize'] = 18
# plt.rcParams['ytick.labelsize'] = 24
plot_kappa_comparison_maps(kappa_recov_avg_fine, pred_kappa_rbf, galaxy_pix1000_unique[mask_20k_gals], nside=64, save_path='images/kappa_comparison_rbf.jpg')
# plt.savefig('images/kappamap_comparison_rbf.jpg', dpi=300)
plot_kappa_comparison_maps(kappa_recov_avg_fine, pred_kappa_dir_rbf, galaxy_pix1000_unique[mask_20k_gals], nside=64, save_path='images/kappa_comparison_directional_rbf.jpg')
# plt.savefig('images/kappamap_comparison_dir_rbf.jpg', dpi=300)

#### Recovery of Statistics

In [ ]:
"""First, we compute the Gaussian statistics of the predicted field.
This involves getting a zero-mean, comparing std, skewness kurtosis."""

from scipy.stats import skew, kurtosis

def compute_gaussian_stats(kappa_field):
    """Compute mean, std, skewness, kurtosis of the kappa field."""
    
    mean = np.nanmean(kappa_field)
    std = np.nanstd(kappa_field)
    skewness = skew(kappa_field, nan_policy='omit')
    kurt = kurtosis(kappa_field, nan_policy='omit')
    print(f"MEAN = {mean:.4e}", f"\nSTD = {std:.4e}", f"\nSkewness = {skewness:.4e}", f"\nKurtosis = {kurt:.4e}")
    print("-"*20)
    return mean#, std, skewness, kurt


In [ ]:
compute_gaussian_stats(pred_kappa_rbf)
compute_gaussian_stats(pred_kappa_dir_rbf)
compute_gaussian_stats(kappa_recov_avg_fine)

In [ ]:
# plt.rcParams['axes.titlesize'] = 24
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 8


fig = plt.figure(figsize=(6, 5))
fig.patch.set_facecolor(bg_color)
plt.hist(kappa_recov_avg_fine, bins=80, density=True, alpha=0.6, label='True Kappa')
plt.hist(pred_kappa_rbf, bins=80, density=True, alpha=0.5, label='Predicted RBF')
plt.vlines(mean, ymin=0, ymax=80, color='black', linestyle='--', label='Predicted Mean', linewidth=2)
# plt.hist(pred_kappa_dir_rbf, bins=100, density=True, alpha=0.5, label='Predicted Dir. RBF'
plt.xlabel(r'$\kappa$')
plt.legend(fontsize=15)
plt.grid()
plt.tight_layout()
plt.savefig('images/kappa_hist_rbf.jpg', dpi=300)
plt.show()

plt.figure()
plt.hist(kappa_recov_avg_fine, bins=80, density=True, alpha=0.6, label='True Kappa')
plt.hist(pred_kappa_dir_rbf, bins=80, density=True, alpha=0.5, label='Predicted Dir. RBF')
# plt.hist(pred_kappa_dir_rbf, bins=100, density=True, alpha=0.5, label='Predicted Dir. RBF'
plt.xlabel(r'$\kappa$')
plt.legend(fontsize=15)
plt.grid()
plt.tight_layout()
plt.savefig('images/kappa_hist_dir_rbf.jpg', dpi=300)
plt.show()


In [ ]:
recov_random_seed = 31
np.random.seed(recov_random_seed)
recovery_kappamap = hp.synfast(cl_kappa_225, nside_fine)
recovery_kappamap = hp.ud_grade(recovery_kappamap, 64)

In [ ]:
import healpy as hp
import numpy as np

pseudo_cl = np.loadtxt("pseudo_cl.txt")

full_map_rbf = np.zeros(hp.nside2npix(64))
full_map_rbf[galaxy_pix1000_unique[mask_20k_gals]] = pred_kappa_rbf

full_map_dir_rbf = np.zeros(hp.nside2npix(64))
full_map_dir_rbf[galaxy_pix1000_unique[mask_20k_gals]] = pred_kappa_dir_rbf

full_map_perf1 = np.zeros(hp.nside2npix(64))
full_map_perf2 = np.zeros(hp.nside2npix(64))
full_map_perf1[galaxy_pix1000_unique[mask_20k_gals]] = kappamap_225[galaxy_pix1000_unique[mask_20k_gals]]
full_map_perf2[galaxy_pix1000_unique[mask_20k_gals]] = recovery_kappamap[galaxy_pix1000_unique[mask_20k_gals]]  # Perfect recovery

# map: 1D array of convergence values, zeros where unobserved
mean_subtracted_rbf = full_map_rbf - np.mean(full_map_rbf[full_map_rbf != 0])   #Remove monopole
cls_raw_rbf = hp.anafast(mean_subtracted_rbf, lmax=256)
f_sky_rbf = np.count_nonzero(mean_subtracted_rbf) / mean_subtracted_rbf.size
print(f_sky_rbf)
cls_corr_rbf = cls_raw_rbf / f_sky_rbf

mean_subtracted_dir_rbf = full_map_dir_rbf - np.mean(full_map_dir_rbf[full_map_dir_rbf != 0])   #Remove monopole
cls_raw_dir_rbf = hp.anafast(mean_subtracted_dir_rbf, lmax=256)
f_sky_dir_rbf = np.count_nonzero(mean_subtracted_dir_rbf) / mean_subtracted_dir_rbf.size
print(f_sky_dir_rbf)
cls_corr_dir_rbf = cls_raw_dir_rbf / f_sky_dir_rbf

mean_subtracted_perf = full_map_perf1 - np.mean(full_map_perf1[full_map_perf1 != 0])   #Remove monopole
cls_raw_perf = hp.anafast(mean_subtracted_perf, lmax=256)
f_sky_perf = np.count_nonzero(mean_subtracted_perf) / mean_subtracted_perf.size
print(f_sky_perf)
cls_corr_perf = cls_raw_perf / f_sky_perf

mean_subtracted_perf2 = full_map_perf2 - np.mean(full_map_perf2[full_map_perf2 != 0])   #Remove monopole
cls_raw_perf2 = hp.anafast(mean_subtracted_perf2, lmax=256)
f_sky_perf2 = np.count_nonzero(mean_subtracted_perf2) / mean_subtracted_perf2.size
print(f_sky_perf2)
cls_corr_perf2 = cls_raw_perf2 / f_sky_perf2


ell = np.arange(len(cls_corr_rbf))
plt.figure(figsize=(6,4))
plt.plot(cls_corr_rbf, label='Recovered Power Spectrum: RBF', color='blue')
plt.plot(cls_corr_dir_rbf, label='Recovered Power Spectrum: Dir. RBF', color='orange')
plt.plot(cls_corr_perf, label='Recovered Kappa Power Spectrum (Perfect)', color='purple')
plt.plot(pseudo_cl, label=r'Pseudo Power Spectrum', color='red')
plt.xlabel(r'$\ell$')
plt.ylabel(r'$C_\ell$')
plt.title('Angular Power Spectrum of Sky Patch')
plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-10, 3*1e-7)
plt.xlim(2, 256)
plt.grid(True, which='both')
plt.tight_layout()
plt.gca().set_aspect('auto', adjustable='box')
plt.legend()
# plt.savefig("images/power_spectrum_comparison.jpg", dpi=300)
plt.show()


### Create new galaxy distributions
We assign each galaxy a new random pixel, such that they are placed somewhere else.
Then, we create a new convergence map (new seed).

In [ ]:
possible_pix64 = galaxy_pix1000_unique
possible_pix1024 = gal_pix_fine_unique
# np.save('possible_pix64.npy', possible_pix64)
# np.save('possible_pix1024.npy', possible_pix1024)

possible_pix64.shape
possible_pix1024.shape


In [ ]:
val_seed = 29
np.random.seed(val_seed)
val_galaxy_pix1000 = np.random.choice(possible_pix1024, size=galaxy_pix1000.size, replace=True)
val_galaxy_pix1000_unique, val_galaxy_pix1000_counts = np.unique(val_galaxy_pix1000, return_counts=True)

val_kappamap = hp.synfast(cl_kappa_225, nside_fine)
val_observed_size = intrinsic_size1000 * (1.0 + val_kappamap[val_galaxy_pix1000])

#### Recover convergence map with new galaxy distribution

In [ ]:
val_seed = 29
np.random.seed(val_seed)
# Draw new random positions of the galaxies
ra_max, ra_min = catalogue1000['ra'].max(), catalogue1000['ra'].min()
dec_max, dec_min = catalogue1000['dec'].max(), catalogue1000['dec'].min()

new_ra = np.random.uniform(ra_min, ra_max, size=catalogue1000.shape[0])
new_dec = np.random.uniform(dec_min, dec_max, size=catalogue1000.shape[0])


# Convert galaxy coordinates to HEALPix pixel indices
galaxy_pix1000_val = hp.ang2pix(nside, new_ra, new_dec, lonlat=True)
galaxy_pix1000_unique_val, galaxy_pix1000_counts_val = np.unique(galaxy_pix1000_val, return_counts=True)

In [ ]:
mask_20k_gals_val = galaxy_pix1000_counts_val >= 20000
mask_20k_gals_val.sum()

In [ ]:
means_val_fine = np.load('mmd2_val_rbf/means_val_fine.npy')
sign_val_fine = np.load('mmd2_val_rbf/sign_val_fine.npy')
mmd2_val_rbf_batch1 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_1.npy")
mmd2_val_rbf_batch2 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_2.npy")
mmd2_val_rbf_batch3 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_3.npy")
mmd2_val_rbf_batch4 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_4.npy")
mmd2_val_rbf_batch5 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_5.npy")
mmd2_val_rbf_batch6 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_6.npy")
mmd2_val_rbf_batch7 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_7.npy")
mmd2_val_rbf_batch8 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_8.npy")
mmd2_val_rbf_batch9 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_9.npy")
mmd2_val_rbf_batch10 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_10.npy")
mmd2_val_rbf_batch11 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_11.npy")
mmd2_val_rbf_batch12 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_12.npy")
mmd2_val_rbf_batch13 = np.load(f"mmd2_val_rbf/mmd2_val_rbf_fine_batch_13.npy")

mmd2_val_rbf = np.concatenate((mmd2_val_rbf_batch1, mmd2_val_rbf_batch2,
                               mmd2_val_rbf_batch3, mmd2_val_rbf_batch4,
                               mmd2_val_rbf_batch5, mmd2_val_rbf_batch6,
                               mmd2_val_rbf_batch7, mmd2_val_rbf_batch8,
                               mmd2_val_rbf_batch9, mmd2_val_rbf_batch10,
                               mmd2_val_rbf_batch11, mmd2_val_rbf_batch12,
                               mmd2_val_rbf_batch13), axis=0)
kappa_val_avg_fine = np.load('mmd2_val_rbf/kappa_val_avg_fine_masked.npy')
# mask_20k_gals_val = val_galaxy_pix1000_counts >= 20000
# kappa_val_avg_fine = kappa_val_avg_fine[mask_20k_gals_val]


signed_kappa2_val = np.sign(kappa_val_avg_fine) * kappa_val_avg_fine**2
signed_mmd2_val_rbf = sign_val_fine * mmd2_val_rbf

In [ ]:
pred_kappa2_val = signed_mmd2_val_rbf/est_rbf_fit[0] - est_rbf_fit[1]/est_rbf_fit[0]
pred_kappa_val = np.sign(pred_kappa2_val) * np.sqrt(np.abs(pred_kappa2_val))

plt.figure()
plt.plot(signed_kappa2_val, pred_kappa2_val, '.', alpha=0.2)
plt.axis('equal')
x = np.array([-0.0004, 0.0004])
plt.plot(x,x, 'b--', label='y=x')
plt.legend()
plt.xlabel(r'True $\kappa^2$')
plt.ylabel(r'Predicted $\kappa^2$ (RBF)')
plt.title('Predicted vs True $\kappa^2$ on Validation Set (RBF)')
plt.tight_layout()
# plt.savefig("images/val_predicted_vs_true_kappa2_rbf.jpg", dpi=300)
plt.show()


In [ ]:
plot_pixel_level_results(kappa_val_avg_fine, pred_kappa_val)

In [ ]:
plot_kappa_comparison_maps(kappa_val_avg_fine, pred_kappa_val, galaxy_pix1000_unique_val[mask_20k_gals_val], nside=64, save_path=None)